使用模型：
POP、itemKNN、FPMC、GRU4Rec、GCE-GNN

数据预处理部分

In [45]:
import sys
sys.path.append(r'./SequentialRecommend01')
import imp

In [46]:
"""自定义模块"""
import preprocess
import utils

import pandas as pd
import numpy as np
imp.reload(preprocess)
imp.reload(utils)

<module 'utils' from './SequentialRecommend01\\utils.py'>

In [47]:
import torch

In [48]:
df_train_cellphone_ = pd.read_csv(r'./SequentialRecommend01/data/Amazon_Cell/train_sessions.csv')

In [49]:
df_train_cellphone_ = preprocess.generate_users(df_train_cellphone_)

In [50]:
#import numpy as np
train_index = np.random.choice(df_train_cellphone_.index, size = int(df_train_cellphone_.shape[0]*0.85))

In [51]:
df_train_cellphone = df_train_cellphone_.loc[train_index, :]
df_valid_cellphone = pd.concat([df_train_cellphone_, df_train_cellphone]).drop_duplicates(keep=False)

In [52]:
#df_train_cellphone = utils.train_test_split()

In [53]:
df_train_cellphone.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39178 entries, 38675 to 1214
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   session  39178 non-null  object
 1   label    39178 non-null  int64 
 2   UserId   39178 non-null  int32 
dtypes: int32(1), int64(1), object(1)
memory usage: 1.0+ MB


In [54]:
cold_laucher = utils.ColdLauncher().run

In [55]:
%%time
df_ordered_cellphone = preprocess.ordered(df_train_cellphone_, user_key='UserId')
n_items = max(df_ordered_cellphone.itemsId.max()+1, 
              df_ordered_cellphone.Next.max()+1
             )
#cellphone_seqs = preprocess.sequence(df_ordered_cellphone, cold_laucher)
#cellphone_seqs = preprocess.sequence(df_ordered_cellphone)
df_ordered_cellphone = preprocess.ordered(df_train_cellphone, user_key='UserId')
cellphone_seqs, cellphone_seqs_next, cellphone_seqs_len, cellphone_max_len  = preprocess.sequence_(df_train_cellphone,
                                                                                                  item_key='session')
cellphone_graph = preprocess.make_graph(df_ordered_cellphone, weights=True)


Wall time: 43.8 s


In [56]:
%%time
unique_items = df_ordered_cellphone.itemsId.unique()
df_train_cellphone.loc[:,'neg_session'] = df_train_cellphone.apply(lambda x:utils.nagetive_sample(unique_items, 
                                                                x.session,
                                                                [x.label]
                                                               ), axis=1)
cellphone_seqs_neg = df_train_cellphone.neg_session.tolist()
"""df_ordered_cellphone = preprocess.ordered(df_train_cellphone,
                                          user_key='UserId',
                                          neg_item_key='neg_session')
neg_cellphone_seqs = preprocess.sequence(df_ordered_cellphone,
                                        item_key='Neg_itemsId',
                                        next_key='Neg_itemsId_Next')"""

Wall time: 1.46 s


"df_ordered_cellphone = preprocess.ordered(df_train_cellphone,\n                                          user_key='UserId',\n                                          neg_item_key='neg_session')\nneg_cellphone_seqs = preprocess.sequence(df_ordered_cellphone,\n                                        item_key='Neg_itemsId',\n                                        next_key='Neg_itemsId_Next')"

In [57]:
print(cellphone_seqs[0:20])
print('-------------------')
print(cellphone_seqs_next[0:20])
print('-------------------')
print(cellphone_seqs_neg[0:20])

[[4481, 456, 0, 0, 0], [490, 0, 0, 0, 0], [724, 0, 0, 0, 0], [2967, 0, 0, 0, 0], [549, 0, 0, 0, 0], [4305, 6817, 0, 0, 0], [723, 0, 0, 0, 0], [903, 7735, 0, 0, 0], [3977, 0, 0, 0, 0], [2206, 0, 0, 0, 0], [2832, 0, 0, 0, 0], [5742, 137, 2782, 0, 0], [5010, 0, 0, 0, 0], [3213, 1743, 0, 0, 0], [5003, 0, 0, 0, 0], [184, 1848, 0, 0, 0], [1137, 489, 0, 0, 0], [4299, 0, 0, 0, 0], [2955, 7541, 0, 0, 0], [3564, 0, 0, 0, 0]]
-------------------
[329, 53, 7007, 5579, 1743, 3694, 723, 226, 4823, 3383, 100, 197, 772, 5876, 2150, 1296, 151, 964, 6947, 6727]
-------------------
[[3344], [7300], [6773], [2639], [536], [4827], [4466], [9582], [4160], [5452], [8353], [2392], [5968], [8677], [608], [4326], [1509], [446], [620], [2246]]


训练模型

In [58]:
import torch
import torch.optim as optim
import torch.nn.functional as F

import FPMC

In [59]:
import configparser
import os

In [60]:
config = configparser.ConfigParser()
config.read(r'./SequentialRecommend01/config.ini')

['./SequentialRecommend01/config.ini']

In [61]:
model_path = r'./SequentialRecommend01/model'

POP, itemsKNN部分

In [62]:
import HOT

session_pop = HOT.SessionPop(item_key='itemsId', support_by_key='sessionId')
session_pop.fit(df_ordered_cellphone)

item_knn = HOT.ItemKNN(session_key='sessionId', item_key='itemsId', time_key='orderId')
item_knn.fit(df_ordered_cellphone)


GRU4Rec部分

In [63]:
import imp

In [64]:
import GRU4Rec
imp.reload(GRU4Rec)

<module 'GRU4Rec' from './SequentialRecommend01\\GRU4Rec.py'>

In [65]:
GRU_config = config['GRU4REC']
for k, v in SEQ_config.items():
    GRU_config[k] = v
for k, v in GRU_config.items():
    print(k , v)

embedding_size 256
hidden_size 512
loss_type BPR
num_layers 2
dropout_prob 0.35
user_id_field UserId
item_id_field itemsId
list_suffix _Seq
item_list_length_field seq_len
next_prefix Next_
neg_prefix Neg_
max_item_list_length max_len


In [66]:

"""GRU_config_new = {}
for k, v in GRU_config.items():
    try:
        GRU_config_new[k.upper()] = int(v)
        print(k)
        continue
    except:
        pass
    try:
        GRU_config_new[k.upper()] = float(v)
        print(k)
        continue
    except:
        pass
    GRU_config_new[k.upper()] = v"""

'GRU_config_new = {}\nfor k, v in GRU_config.items():\n    try:\n        GRU_config_new[k.upper()] = int(v)\n        print(k)\n        continue\n    except:\n        pass\n    try:\n        GRU_config_new[k.upper()] = float(v)\n        print(k)\n        continue\n    except:\n        pass\n    GRU_config_new[k.upper()] = v'

In [67]:
print(len(cellphone_seqs))
print(len(cellphone_seqs_len))
print(len(cellphone_seqs_next))
print(len(cellphone_seqs_neg))

39178
39178
39178
39178


In [68]:
cellphone_seqs = utils.to_tensor(cellphone_seqs)
cellphone_seqs_len = utils.to_tensor(cellphone_seqs_len).unsqueeze(1)
cellphone_seqs_next = utils.to_tensor(cellphone_seqs_next).unsqueeze(1)
cellphone_seqs_neg = utils.to_tensor(cellphone_seqs_neg)

In [69]:
session_id = list(range(len(cellphone_seqs)))
#session_id = utils.to_tensor(session_id)

In [70]:
Interaction_GRU_all = []
item_seqs_name = GRU_config['ITEM_ID_FIELD'] + GRU_config['LIST_SUFFIX']
next_item_seqs_name = GRU_config['NEXT_PREFIX'] + GRU_config['ITEM_ID_FIELD']
negative_seqs_name = GRU_config['NEG_PREFIX'] + GRU_config['ITEM_ID_FIELD']
#for session_batch in utils.batch_loader(session_id, int(BASE_config['batch_size'])):
for session_batch in utils.batch_loader(session_id, 128):
    session_batch = utils.to_tensor(session_batch)
    Interaction_GRU = {}
    Interaction_GRU[item_seqs_name] = cellphone_seqs[session_batch]
    Interaction_GRU[GRU_config['item_list_length_field']] = cellphone_seqs_len[session_batch]
    Interaction_GRU[next_item_seqs_name] = cellphone_seqs_next[session_batch]
    Interaction_GRU[negative_seqs_name] = cellphone_seqs_neg[session_batch]
    Interaction_GRU_all.append(Interaction_GRU)

In [71]:
GRU_config['device'] = 'cuda:0'

In [72]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [73]:
GRU_config['loss_type '] = 'CE'

In [74]:
GRU_model = GRU4Rec.GRU4Rec(GRU_config, df_ordered_cellphone).to(device)
optimizer = optim.Adam(GRU_model.parameters(), lr = 0.0002)
GRU_model.train()
for i in range(80):
    loss_ = 0
    #count = 0
    for batch in Interaction_GRU_all:
        items_seqs = batch[item_seqs_name]
        items_seqs_len = batch[GRU_config['item_list_length_field']]
        #batch[next_item_seqs_name] = items_next
        #batch[negative_seqs_name] = neg_next
        GRU_model(items_seqs, items_seqs_len)
        loss = GRU_model.calculate_loss(batch)
        loss_+=loss
        #count+=1
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if i>0 and (i+1)%5 == 0:
        torch.save(GRU_model, os.path.join(model_path, 'GRU_{}.pth'.format(i+1)))
    print(f'epoch {i+1}: ', 'loss: ', loss_/len(Interaction_GRU_all))

epoch 1:  loss:  tensor(0.5941, device='cuda:0', grad_fn=<DivBackward0>)
epoch 2:  loss:  tensor(0.3994, device='cuda:0', grad_fn=<DivBackward0>)
epoch 3:  loss:  tensor(0.3544, device='cuda:0', grad_fn=<DivBackward0>)
epoch 4:  loss:  tensor(0.3345, device='cuda:0', grad_fn=<DivBackward0>)
epoch 5:  loss:  tensor(0.3185, device='cuda:0', grad_fn=<DivBackward0>)
epoch 6:  loss:  tensor(0.3064, device='cuda:0', grad_fn=<DivBackward0>)
epoch 7:  loss:  tensor(0.2892, device='cuda:0', grad_fn=<DivBackward0>)
epoch 8:  loss:  tensor(0.2765, device='cuda:0', grad_fn=<DivBackward0>)
epoch 9:  loss:  tensor(0.2658, device='cuda:0', grad_fn=<DivBackward0>)
epoch 10:  loss:  tensor(0.2570, device='cuda:0', grad_fn=<DivBackward0>)
epoch 11:  loss:  tensor(0.2517, device='cuda:0', grad_fn=<DivBackward0>)
epoch 12:  loss:  tensor(0.2498, device='cuda:0', grad_fn=<DivBackward0>)
epoch 13:  loss:  tensor(0.2460, device='cuda:0', grad_fn=<DivBackward0>)
epoch 14:  loss:  tensor(0.2443, device='cuda:0

SRGNN部分

In [75]:
import SRGNN
imp.reload(SRGNN)

<module 'SRGNN' from './SequentialRecommend01\\SRGNN.py'>

In [76]:
GRU_config['step'] = '1'

In [77]:
srgnn_model = SRGNN.SRGNN(GRU_config, df_ordered_cellphone).to(device)
optimizer = optim.Adam(srgnn_model.parameters(), lr = 0.0002)
srgnn_model.train()
for i in range(80):
    loss_ = 0
    #count = 0
    for batch in Interaction_GRU_all:
        items_seqs = batch[item_seqs_name]
        items_seqs_len = batch[GRU_config['item_list_length_field']]
        #batch[next_item_seqs_name] = items_next
        #batch[negative_seqs_name] = neg_next
        srgnn_model(items_seqs, items_seqs_len)
        loss = srgnn_model.calculate_loss(batch)
        loss_+=loss
        #count+=1
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if i>0 and (i+1)%5 == 0:
            torch.save(srgnn_model, os.path.join(model_path, 'SRGNN_{}.pth'.format(i+1)))
    print(f'epoch {i+1}: ', 'loss: ', loss_/len(Interaction_GRU_all))

epoch 1:  loss:  tensor(0.5676, device='cuda:0', grad_fn=<DivBackward0>)
epoch 2:  loss:  tensor(0.3970, device='cuda:0', grad_fn=<DivBackward0>)
epoch 3:  loss:  tensor(0.3486, device='cuda:0', grad_fn=<DivBackward0>)
epoch 4:  loss:  tensor(0.3112, device='cuda:0', grad_fn=<DivBackward0>)
epoch 5:  loss:  tensor(0.2783, device='cuda:0', grad_fn=<DivBackward0>)
epoch 6:  loss:  tensor(0.2653, device='cuda:0', grad_fn=<DivBackward0>)
epoch 7:  loss:  tensor(0.2708, device='cuda:0', grad_fn=<DivBackward0>)
epoch 8:  loss:  tensor(0.2414, device='cuda:0', grad_fn=<DivBackward0>)
epoch 9:  loss:  tensor(0.2190, device='cuda:0', grad_fn=<DivBackward0>)
epoch 10:  loss:  tensor(0.1919, device='cuda:0', grad_fn=<DivBackward0>)
epoch 11:  loss:  tensor(0.1954, device='cuda:0', grad_fn=<DivBackward0>)
epoch 12:  loss:  tensor(0.1785, device='cuda:0', grad_fn=<DivBackward0>)
epoch 13:  loss:  tensor(0.1650, device='cuda:0', grad_fn=<DivBackward0>)
epoch 14:  loss:  tensor(0.1604, device='cuda:0

In [78]:
srgnn_model.device

'cuda:0'

NARM部分

In [79]:
for k, v in GRU_config.items():
    print(k, v)

embedding_size 256
hidden_size 512
loss_type BPR
num_layers 2
dropout_prob 0.35
user_id_field UserId
item_id_field itemsId
list_suffix _Seq
item_list_length_field seq_len
next_prefix Next_
neg_prefix Neg_
max_item_list_length max_len
device cuda:0
loss_type  CE
step 1


In [80]:
import narm
import torch.nn as nn

In [81]:
def len_argsort(seq):
    return sorted(range(len(seq)), key=lambda x: len(seq[x]), reverse=True)

In [82]:
#df_train_cellphone.loc[len_argsort(df_train_cellphone.session.tolist()),'session']

In [83]:
df_train_cellphone.columns

Index(['session', 'label', 'UserId', 'session_len', 'session_pad',
       'neg_session'],
      dtype='object')

In [84]:
imp.reload(preprocess)
imp.reload(narm)

<module 'narm' from './SequentialRecommend01\\narm.py'>

In [85]:
print(df_train_cellphone.shape[0])
len(len_argsort(df_train_cellphone.session.tolist()))

39178


39178

In [86]:
n_items = max(df_ordered_cellphone.itemsId.max()+1, 
              df_ordered_cellphone.Next.max()+1
             )
hidden_size = 512
embedding_dim = 256
batch_size = 128
n_layers = 2
loss_func = nn.CrossEntropyLoss()

narm_model = narm.NARM(n_items, hidden_size, embedding_dim, batch_size, n_layers).to(device)
narm_model.train()
optimizer = optim.Adam(narm_model.parameters(), lr = 0.0002)
max_len = df_train_cellphone.session_len.max()
df_train_cellphone.reset_index(drop=True, inplace=True)
for i in range(120):
    loss_ = 0
    count = 0
    for batch in utils.batch_loader(df_train_cellphone.index.tolist(), 128):
        
        df_chunk = df_train_cellphone.loc[batch,:]
        
        index_ = len_argsort(df_chunk.session.tolist())
        
        df_chunk.reset_index(drop=True, inplace=True)
        #print(df_chunk.loc[index_,:].head())
        items_seqs, items_next, items_seqs_len, _ = preprocess.sequence_(df_chunk.loc[index_,:],
                                                                     item_key='session',
                                                                     next_key='label')
        items_seqs = utils.to_tensor(items_seqs).T
        #print(items_seqs.shape)
        items_seqs_len = utils.to_tensor(items_seqs_len).to(torch.device('cpu'))
        items_next = utils.to_tensor(items_next)
        #print(items_next.shape)
        #batch[negative_seqs_name] = neg_next
        output = narm_model(items_seqs, items_seqs_len)
        loss = loss_func(output, items_next)
        loss_+=loss
        count+=1
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if i>0 and (i+1)%5 == 0:
        torch.save(narm_model, os.path.join(model_path, 'NARM_{}.pth'.format(i+1)))
    print(f'epoch {i+1}: ', 'loss: ', loss_/count)

epoch 1:  loss:  tensor(9.6200, device='cuda:0', grad_fn=<DivBackward0>)
epoch 2:  loss:  tensor(8.8536, device='cuda:0', grad_fn=<DivBackward0>)
epoch 3:  loss:  tensor(8.4551, device='cuda:0', grad_fn=<DivBackward0>)
epoch 4:  loss:  tensor(8.1330, device='cuda:0', grad_fn=<DivBackward0>)
epoch 5:  loss:  tensor(7.8285, device='cuda:0', grad_fn=<DivBackward0>)
epoch 6:  loss:  tensor(7.5640, device='cuda:0', grad_fn=<DivBackward0>)
epoch 7:  loss:  tensor(7.3371, device='cuda:0', grad_fn=<DivBackward0>)
epoch 8:  loss:  tensor(7.1716, device='cuda:0', grad_fn=<DivBackward0>)
epoch 9:  loss:  tensor(6.9163, device='cuda:0', grad_fn=<DivBackward0>)
epoch 10:  loss:  tensor(6.6279, device='cuda:0', grad_fn=<DivBackward0>)
epoch 11:  loss:  tensor(6.3930, device='cuda:0', grad_fn=<DivBackward0>)
epoch 12:  loss:  tensor(6.1502, device='cuda:0', grad_fn=<DivBackward0>)
epoch 13:  loss:  tensor(5.9363, device='cuda:0', grad_fn=<DivBackward0>)
epoch 14:  loss:  tensor(5.7159, device='cuda:0

epoch 112:  loss:  tensor(1.7334, device='cuda:0', grad_fn=<DivBackward0>)
epoch 113:  loss:  tensor(1.7327, device='cuda:0', grad_fn=<DivBackward0>)
epoch 114:  loss:  tensor(1.7313, device='cuda:0', grad_fn=<DivBackward0>)
epoch 115:  loss:  tensor(1.7159, device='cuda:0', grad_fn=<DivBackward0>)
epoch 116:  loss:  tensor(1.7251, device='cuda:0', grad_fn=<DivBackward0>)
epoch 117:  loss:  tensor(1.7045, device='cuda:0', grad_fn=<DivBackward0>)
epoch 118:  loss:  tensor(1.7118, device='cuda:0', grad_fn=<DivBackward0>)
epoch 119:  loss:  tensor(1.7039, device='cuda:0', grad_fn=<DivBackward0>)
epoch 120:  loss:  tensor(1.6932, device='cuda:0', grad_fn=<DivBackward0>)


In [87]:
print(narm_model.n_items)


9805


In [88]:
output.shape

torch.Size([128, 9805])

In [89]:
items_next.shape

torch.Size([128])

In [90]:
items_seqs.shape

torch.Size([5, 128])

检验模型

POP和itemKNN部分

In [91]:
df_valid_cellphone.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19686 entries, 3 to 46091
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   session  19686 non-null  object
 1   label    19686 non-null  int64 
 2   UserId   19686 non-null  int32 
dtypes: int32(1), int64(1), object(1)
memory usage: 538.3+ KB


In [92]:
df_ordered_cellphone_valid = preprocess.ordered(df_valid_cellphone, user_key='UserId')
cellphone_seqs_valid, cellphone_seqs_next_valid, cellphone_seqs_len_valid, _ = preprocess.sequence_(df_valid_cellphone,
                                                                                                  item_key='session')

In [93]:
df_ordered_cellphone_valid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32693 entries, 0 to 32692
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   sessionId  32693 non-null  int64
 1   itemsId    32693 non-null  int64
 2   Next       32693 non-null  int64
 3   orderId    32693 non-null  int64
 4   UserId     32693 non-null  int32
dtypes: int32(1), int64(4)
memory usage: 1.1 MB


In [94]:
session_pop_result = []
itemsKNN_result = []
for session_id in df_ordered_cellphone_valid.sessionId.unique():
    items_id = df_ordered_cellphone_valid[df_ordered_cellphone_valid.sessionId == session_id].itemsId.tolist()
    for itemId in items_id:
        predict_items = np.random.choice(df_ordered_cellphone.itemsId.unique(), size=10)
        predict_items_ = np.random.choice(session_pop.pop_list.index, size=10)
        predict_items = np.concatenate([predict_items, predict_items_])
        
        result = session_pop.predict_next(session_id, itemId, predict_items)
        session_pop_result.append(result)
        
        #predict_items = np.random.choice(df_ordered_cellphone.itemsId.unique(), size=10)
        predict_items_ = np.random.choice(list(item_knn.sims.keys()), size=20)
        #predict_items = np.concatenate([predict_items, predict_items_])
        try:
            result2 = item_knn.predict_next(session_id, itemId, predict_items)
            itemsKNN_result.append(result2)
        except:
            itemsKNN_result.append([])
            continue

In [95]:
session_pop.pop_list.shape

(100,)

In [96]:
session_pop_result[0:5]

[7044    0.000000
 7306    0.000000
 860     0.000000
 8671    0.000000
 3688    0.000000
 508     0.000000
 7643    0.000000
 492     0.000000
 6233    0.000000
 1820    0.000000
 153     0.989130
 13      0.994444
 1458    0.989362
 348     0.986667
 342     0.993056
 1185    0.990566
 196     0.992424
 265     0.994083
 206     0.985507
 1913    0.987013
 dtype: float64,
 3032    0.000000
 5394    0.000000
 2948    0.000000
 7457    0.000000
 3353    0.000000
 5140    0.000000
 4643    0.000000
 2701    0.000000
 7099    0.000000
 9650    0.000000
 1143    0.988636
 528     0.994924
 1185    0.990566
 61      0.987805
 528     0.994924
 18      0.996350
 319     0.984615
 342     0.993056
 331     0.993243
 7       0.991379
 dtype: float64,
 6200    0.000000
 8048    0.000000
 9645    0.000000
 6383    0.000000
 1277    0.000000
 2877    0.000000
 6445    0.000000
 7714    0.000000
 6200    0.000000
 6966    0.000000
 279     0.990196
 568     0.986667
 165     0.996350
 251     0.9

GRU部分

In [97]:
import torch.nn as nn

In [98]:
GRU_model = torch.load(r'./SequentialRecommend01/model/GRU_80.pth')

In [99]:
cellphone_seqs_valid = utils.to_tensor(cellphone_seqs_valid)
cellphone_seqs_len_valid = utils.to_tensor(cellphone_seqs_len_valid).unsqueeze(1)
cellphone_seqs_next_valid = utils.to_tensor(cellphone_seqs_next_valid).unsqueeze(1)
#cellphone_seqs_neg = utils.to_tensor(cellphone_seqs_neg)


In [100]:
session_id = list(range(len(cellphone_seqs_valid)))

In [101]:
Interaction_GRU_valid = []
item_seqs_name = GRU_config['ITEM_ID_FIELD'] + GRU_config['LIST_SUFFIX']
next_item_seqs_name = GRU_config['NEXT_PREFIX'] + GRU_config['ITEM_ID_FIELD']
#negative_seqs_name = GRU_config['NEG_PREFIX'] + GRU_config['ITEM_ID_FIELD']
#for session_batch in utils.batch_loader(session_id, int(BASE_config['batch_size'])):
for session_batch in utils.batch_loader(session_id, 128):
    session_batch = utils.to_tensor(session_batch)
    Interaction_GRU = {}
    Interaction_GRU[item_seqs_name] = cellphone_seqs_valid[session_batch]
    Interaction_GRU[GRU_config['item_list_length_field']] = cellphone_seqs_len_valid[session_batch]
    Interaction_GRU[next_item_seqs_name] = cellphone_seqs_next_valid[session_batch]
    #Interaction_GRU[negative_seqs_name] = cellphone_seqs_neg[session_batch]
    Interaction_GRU_valid.append(Interaction_GRU)

In [102]:
import math

class CosineSimilarity(nn.Module):
    """
    This similarity function simply computes the cosine similarity between each pair of vectors. It has
    no parameters.
    """
    def forward(self, tensor_1, tensor_2):
        normalized_tensor_1 = tensor_1 / tensor_1.norm(dim=-1, keepdim=True)
        normalized_tensor_2 = tensor_2 / tensor_2.norm(dim=-1, keepdim=True)
        return (normalized_tensor_1 * normalized_tensor_2).sum(dim=-1)
    
class DotProductSimilarity(nn.Module):
    """
    This similarity function simply computes the dot product between each pair of vectors, with an
    optional scaling to reduce the variance of the output elements.
    """
    def __init__(self, scale_output=False):
        super(DotProductSimilarity, self).__init__()
        self.scale_output = scale_output
    def forward(self, tensor_1, tensor_2):
        result = (tensor_1 * tensor_2).sum(dim=-1)
        if self.scale_output:
            # TODO why allennlp do multiplication at here ?
            result /= math.sqrt(tensor_1.size(-1))
        return result



In [103]:
import torch.nn.functional as F
import metric

In [104]:
GRU_model.eval()
GRU_result = []
sims = 0.0
count = 0
cosine_similarity = CosineSimilarity()
product_similarity = DotProductSimilarity()
recalls = []
mrrs = []
with torch.no_grad():
    for batch in Interaction_GRU_valid:
        items_seq = batch[item_seqs_name]
        items_seq_len = batch[GRU_config['item_list_length_field']]
        next_item_seqs = batch[next_item_seqs_name]
        output = GRU_model(items_seq, items_seq_len)
        #nexts = GRU_model.item_embedding(next_item_seqs).view(-1, 256)
        sims = torch.matmul(output, GRU_model.item_embedding.weight.T)
        #print(sims.shape)
        logits = F.softmax(sims, dim = 1)
        #print(logits.shape)
        #sims += cosine_similarity(output, nexts)
        #sims = torch.cosine_similarity(output, nexts)
        recall, mrr = metric.evaluate(logits, next_item_seqs, k = 20)
        recalls.append(recall)
        mrrs.append(mrr)
        #GRU_result.append(output)
        #count+=1
#sims/count
print(np.mean(recalls), np.mean(mrrs))

0.015016233766233766 0.0064277421959370445


In [105]:
(sims/count).mean()

tensor(nan, device='cuda:0')

In [106]:
torch.cuda.empty_cache()

SRGNN部分

In [107]:
SRGNN_model = torch.load(r'./SequentialRecommend01/model/SRGNN_50.pth')

In [108]:
SRGNN_model.eval()
sims = 0.0
count = 0

with torch.no_grad():
    for batch in Interaction_GRU_valid:
        items_seq = batch[item_seqs_name]
        items_seq_len = batch[GRU_config['item_list_length_field']]
        next_item_seqs = batch[next_item_seqs_name]
        output = SRGNN_model(items_seq, items_seq_len)
        nexts = SRGNN_model.item_embedding(next_item_seqs).view(-1, 256)
        sims += cosine_similarity(output, nexts)
        #sims = torch.cosine_similarity(output, nexts)
        
        count+=1
sims/count

tensor([0.0882, 0.1246, 0.0713, 0.1051, 0.0893, 0.1176, 0.0757, 0.0700, 0.0503,
        0.0320, 0.0643, 0.1234, 0.1146, 0.0931, 0.1022, 0.1223, 0.0678, 0.0559,
        0.0995, 0.0937, 0.0994, 0.0865, 0.0829, 0.0613, 0.0730, 0.1256, 0.0938,
        0.0818, 0.1192, 0.0605, 0.0772, 0.0987, 0.0691, 0.0937, 0.1294, 0.0786,
        0.1212, 0.1331, 0.1241, 0.0418, 0.0630, 0.1041, 0.1436, 0.0625, 0.0785,
        0.1442, 0.1359, 0.1200, 0.0125, 0.1318, 0.1010, 0.0559, 0.0763, 0.0831,
        0.1147, 0.1555, 0.0828, 0.1152, 0.0591, 0.1124, 0.0782, 0.0874, 0.0194,
        0.1221, 0.1143, 0.1027, 0.1119, 0.1424, 0.0936, 0.0877, 0.1195, 0.0922,
        0.1009, 0.1339, 0.0545, 0.1198, 0.0696, 0.0462, 0.0959, 0.0536, 0.1153,
        0.1190, 0.1065, 0.1364, 0.1072, 0.0827, 0.1025, 0.0873, 0.0730, 0.1149,
        0.0708, 0.1349, 0.1408, 0.0768, 0.0888, 0.1046, 0.1196, 0.0504, 0.0997,
        0.0808, 0.1303, 0.0802, 0.1881, 0.0703, 0.0649, 0.1213, 0.1340, 0.0824,
        0.0860, 0.0830, 0.0894, 0.0470, 

In [109]:
(sims/count).mean()

tensor(0.0939, device='cuda:0')

NARM部分

In [110]:
NARM_model = torch.load(r'./SequentialRecommend01/model/NARM_120.pth')

In [111]:
NARM_model.eval()
sims = 0.0
count = 0
topk = 20
recalls = []
mrrs = []
with torch.no_grad():
    df_valid_cellphone.reset_index(drop=True, inplace=True)
    
        
    for batch in utils.batch_loader(df_valid_cellphone.index.tolist(), 128):

        df_chunk = df_valid_cellphone.loc[batch,:]

        index_ = len_argsort(df_chunk.session.tolist())

        df_chunk.reset_index(drop=True, inplace=True)
        items_seqs, items_next, items_seqs_len, _ = preprocess.sequence_(df_chunk.loc[index_,:],
                                                                     item_key='session',
                                                                     next_key='label')
        items_seqs = utils.to_tensor(items_seqs).T
        #print(items_seqs.shape)
        items_seqs_len = utils.to_tensor(items_seqs_len).to(torch.device('cpu'))
        items_next = utils.to_tensor(items_next)
        #print(items_next.shape)
        #batch[negative_seqs_name] = neg_next
        outputs = NARM_model(items_seqs, items_seqs_len)
        logits = F.softmax(outputs, dim = 1)
        #print(logits.shape, items_next.shape)
        recall, mrr = metric.evaluate(logits, items_next, k = topk)
        recalls.append(recall)
        mrrs.append(mrr)


In [112]:
np.mean(recall)

0.0625

In [113]:
np.mean(mrrs)

0.045414336557899206

In [114]:
items_next

tensor([ 454,  449, 7737,   66, 8731,  454, 1853, 6208, 1827, 3912, 5329, 2099,
        8175, 8269,  291, 1546, 6920,  104, 7128, 5645, 9286, 2551, 2456, 1558,
        6515, 7270, 1360, 6603, 8367, 1190,  959,  918,  571,  450,  252, 7789,
        2600, 8218,  362, 2011, 2842, 2513, 5896,  958, 6828, 5226, 3322,  833,
        1407,  480, 1953,  304, 7748,    8, 6125, 1892, 7346, 7501, 5427, 5101,
        1046, 4813,  993, 3123,  697, 1281, 1388, 8403, 1021, 7258, 7967,  723,
        5277, 1996, 2672,   92,  812, 6459, 5013, 1276,   36,  398, 1655, 1156,
        3805,  358, 1142, 1041, 2856,  454, 3241, 3988, 5897, 2310, 2827, 5554,
         539, 3219, 8510, 5358, 7809, 8330,  840,  368, 4923, 1653, 3822,   53,
        6641, 2485,  270,  594, 9382, 8484, 2287,  450, 1929, 8068, 8486, 3916,
        8540,  547,  273, 2362, 3865, 8393, 4566, 2607], device='cuda:0')

In [115]:
df_train_cellphone_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46092 entries, 0 to 46091
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   session  46092 non-null  object
 1   label    46092 non-null  int64 
 2   UserId   46092 non-null  int32 
dtypes: int32(1), int64(1), object(1)
memory usage: 900.4+ KB


In [116]:
print(df_ordered_cellphone.itemsId.max())
print(df_ordered_cellphone_valid.itemsId.max())

9803
9788


In [117]:
print(cellphone_seqs_len.max())
print(cellphone_seqs_len_valid.max())

tensor(5, device='cuda:0')
tensor(5, device='cuda:0')


In [118]:
df_beauty = pd.read_csv(r'./SequentialRecommend01/data/Amazon_Beauty/train_sessions.csv')

In [119]:
df_beauty.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50297 entries, 0 to 50296
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   session  50297 non-null  object
 1   label    50297 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 786.0+ KB


In [120]:
df_ordered_beauty = preprocess.ordered(df_beauty)

In [121]:
df_ordered_beauty.itemsId.max()

11673

In [122]:
df_ordered_beauty.Next.max()

11672

In [123]:
beauty_seqs, beauty_seqs_next, beauty_seqs_len, beauty_max_len  = preprocess.sequence_(df_beauty,
                                                                                       item_key='session')
beauty_graph = preprocess.make_graph(df_ordered_beauty, weights=True)

In [124]:
import narm
imp.reload(narm)

<module 'narm' from './SequentialRecommend01\\narm.py'>

In [125]:
max(beauty_seqs_len)

5

In [126]:
df_ordered_cellphone_ = preprocess.ordered(df_train_cellphone_)

In [127]:
n_items = max(df_ordered_cellphone_.itemsId.max()+1, 
              df_ordered_cellphone_.Next.max()+1
             )
hidden_size = 512
embedding_dim = 256
batch_size = 128
n_layers = 2
loss_func = nn.CrossEntropyLoss()

narm_model = narm.NARM(n_items, hidden_size, embedding_dim, batch_size, n_layers).to(device)
narm_model.train()
optimizer = optim.Adam(narm_model.parameters(), lr = 0.0002)
max_len = 5
df_train_cellphone_.reset_index(drop=True, inplace=True)
for i in range(150):
    loss_ = 0
    count = 0
    for batch in utils.batch_loader(df_train_cellphone_.index.tolist(), 128):
        
        df_chunk = df_train_cellphone_.loc[batch,:]
        
        index_ = len_argsort(df_chunk.session.tolist())
        
        df_chunk.reset_index(drop=True, inplace=True)
        df_check = df_chunk.loc[index_,:]
        #print(df_check.head())
        items_seqs, items_next, items_seqs_len, _ = preprocess.sequence_(df_check,
                                                                     item_key='session',
                                                                     next_key='label')
        items_seqs = utils.to_tensor(items_seqs).T
        #print(items_seqs.shape)
        items_seqs_len = utils.to_tensor(items_seqs_len).to(torch.device('cpu'))
        items_next = utils.to_tensor(items_next)
        #print(items_next.shape)
        #batch[negative_seqs_name] = neg_next
        output = narm_model(items_seqs, items_seqs_len)
        loss = loss_func(output, items_next)
        loss_+=loss
        count+=1
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if i>0 and (i+1)%5 == 0:
        torch.save(narm_model, os.path.join(model_path, 'NARM_cellphone_{}.pth'.format(i+1)))
    print(f'epoch {i+1}: ', 'loss: ', loss_/count)

epoch 1:  loss:  tensor(9.6484, device='cuda:0', grad_fn=<DivBackward0>)
epoch 2:  loss:  tensor(9.0857, device='cuda:0', grad_fn=<DivBackward0>)
epoch 3:  loss:  tensor(8.8525, device='cuda:0', grad_fn=<DivBackward0>)
epoch 4:  loss:  tensor(8.6778, device='cuda:0', grad_fn=<DivBackward0>)
epoch 5:  loss:  tensor(8.5297, device='cuda:0', grad_fn=<DivBackward0>)
epoch 6:  loss:  tensor(8.3798, device='cuda:0', grad_fn=<DivBackward0>)
epoch 7:  loss:  tensor(8.2508, device='cuda:0', grad_fn=<DivBackward0>)
epoch 8:  loss:  tensor(8.1274, device='cuda:0', grad_fn=<DivBackward0>)
epoch 9:  loss:  tensor(7.9976, device='cuda:0', grad_fn=<DivBackward0>)
epoch 10:  loss:  tensor(7.8647, device='cuda:0', grad_fn=<DivBackward0>)
epoch 11:  loss:  tensor(7.7326, device='cuda:0', grad_fn=<DivBackward0>)
epoch 12:  loss:  tensor(7.5982, device='cuda:0', grad_fn=<DivBackward0>)
epoch 13:  loss:  tensor(7.4581, device='cuda:0', grad_fn=<DivBackward0>)
epoch 14:  loss:  tensor(7.3204, device='cuda:0

epoch 112:  loss:  tensor(2.6970, device='cuda:0', grad_fn=<DivBackward0>)
epoch 113:  loss:  tensor(2.6675, device='cuda:0', grad_fn=<DivBackward0>)
epoch 114:  loss:  tensor(2.6559, device='cuda:0', grad_fn=<DivBackward0>)
epoch 115:  loss:  tensor(2.6393, device='cuda:0', grad_fn=<DivBackward0>)
epoch 116:  loss:  tensor(2.6325, device='cuda:0', grad_fn=<DivBackward0>)
epoch 117:  loss:  tensor(2.6305, device='cuda:0', grad_fn=<DivBackward0>)
epoch 118:  loss:  tensor(2.6135, device='cuda:0', grad_fn=<DivBackward0>)
epoch 119:  loss:  tensor(2.5968, device='cuda:0', grad_fn=<DivBackward0>)
epoch 120:  loss:  tensor(2.5981, device='cuda:0', grad_fn=<DivBackward0>)
epoch 121:  loss:  tensor(2.5792, device='cuda:0', grad_fn=<DivBackward0>)
epoch 122:  loss:  tensor(2.5720, device='cuda:0', grad_fn=<DivBackward0>)
epoch 123:  loss:  tensor(2.5624, device='cuda:0', grad_fn=<DivBackward0>)
epoch 124:  loss:  tensor(2.5447, device='cuda:0', grad_fn=<DivBackward0>)
epoch 125:  loss:  tensor

In [128]:
df_ordered_beauty.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90154 entries, 0 to 90153
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   sessionId  90154 non-null  int64
 1   itemsId    90154 non-null  int64
 2   Next       90154 non-null  int64
 3   orderId    90154 non-null  int64
dtypes: int64(4)
memory usage: 2.8 MB


In [129]:
n_items = max(df_ordered_beauty.itemsId.max()+1, 
              df_ordered_beauty.Next.max()+1
             )
hidden_size = 512
embedding_dim = 256
batch_size = 128
n_layers = 2
loss_func = nn.CrossEntropyLoss()

narm_model = narm.NARM(n_items, hidden_size, embedding_dim, batch_size, n_layers).to(device)
narm_model.train()
optimizer = optim.Adam(narm_model.parameters(), lr = 0.0002)
max_len = 5
df_beauty.reset_index(drop=True, inplace=True)
for i in range(180):
    loss_ = 0
    count = 0
    for batch in utils.batch_loader(df_beauty.index.tolist(), 128):
        
        df_chunk = df_beauty.loc[batch,:]
        
        index_ = len_argsort(df_chunk.session.tolist())
        
        df_chunk.reset_index(drop=True, inplace=True)
        df_check = df_chunk.loc[index_,:]
        #print(df_check.head())
        items_seqs, items_next, items_seqs_len, _ = preprocess.sequence_(df_check,
                                                                     item_key='session',
                                                                     next_key='label')
        items_seqs = utils.to_tensor(items_seqs).T
        #print(items_seqs.shape)
        items_seqs_len = utils.to_tensor(items_seqs_len).to(torch.device('cpu'))
        items_next = utils.to_tensor(items_next)
        #print(items_next.shape)
        #batch[negative_seqs_name] = neg_next
        output = narm_model(items_seqs, items_seqs_len)
        loss = loss_func(output, items_next)
        loss_+=loss
        count+=1
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if i>0 and (i+1)%5 == 0:
        torch.save(narm_model, os.path.join(model_path, 'NARM_beauty_{}.pth'.format(i+1)))
    print(f'epoch {i+1}: ', 'loss: ', loss_/count)

epoch 1:  loss:  tensor(9.7018, device='cuda:0', grad_fn=<DivBackward0>)
epoch 2:  loss:  tensor(9.0132, device='cuda:0', grad_fn=<DivBackward0>)
epoch 3:  loss:  tensor(8.7306, device='cuda:0', grad_fn=<DivBackward0>)
epoch 4:  loss:  tensor(8.5434, device='cuda:0', grad_fn=<DivBackward0>)
epoch 5:  loss:  tensor(8.3578, device='cuda:0', grad_fn=<DivBackward0>)
epoch 6:  loss:  tensor(8.2025, device='cuda:0', grad_fn=<DivBackward0>)
epoch 7:  loss:  tensor(8.0526, device='cuda:0', grad_fn=<DivBackward0>)
epoch 8:  loss:  tensor(7.8944, device='cuda:0', grad_fn=<DivBackward0>)
epoch 9:  loss:  tensor(7.7583, device='cuda:0', grad_fn=<DivBackward0>)
epoch 10:  loss:  tensor(7.6147, device='cuda:0', grad_fn=<DivBackward0>)
epoch 11:  loss:  tensor(7.4842, device='cuda:0', grad_fn=<DivBackward0>)
epoch 12:  loss:  tensor(7.3450, device='cuda:0', grad_fn=<DivBackward0>)
epoch 13:  loss:  tensor(7.2115, device='cuda:0', grad_fn=<DivBackward0>)
epoch 14:  loss:  tensor(7.0727, device='cuda:0

epoch 112:  loss:  tensor(2.3835, device='cuda:0', grad_fn=<DivBackward0>)
epoch 113:  loss:  tensor(2.3642, device='cuda:0', grad_fn=<DivBackward0>)
epoch 114:  loss:  tensor(2.3502, device='cuda:0', grad_fn=<DivBackward0>)
epoch 115:  loss:  tensor(2.3325, device='cuda:0', grad_fn=<DivBackward0>)
epoch 116:  loss:  tensor(2.3272, device='cuda:0', grad_fn=<DivBackward0>)
epoch 117:  loss:  tensor(2.3289, device='cuda:0', grad_fn=<DivBackward0>)
epoch 118:  loss:  tensor(2.3167, device='cuda:0', grad_fn=<DivBackward0>)
epoch 119:  loss:  tensor(2.2984, device='cuda:0', grad_fn=<DivBackward0>)
epoch 120:  loss:  tensor(2.2804, device='cuda:0', grad_fn=<DivBackward0>)
epoch 121:  loss:  tensor(2.2804, device='cuda:0', grad_fn=<DivBackward0>)
epoch 122:  loss:  tensor(2.2663, device='cuda:0', grad_fn=<DivBackward0>)
epoch 123:  loss:  tensor(2.2550, device='cuda:0', grad_fn=<DivBackward0>)
epoch 124:  loss:  tensor(2.2447, device='cuda:0', grad_fn=<DivBackward0>)
epoch 125:  loss:  tensor

In [130]:
df_cellphhone_test = pd.read_csv(r'./SequentialRecommend01/data/Amazon_Cell/test_sessions.csv')
df_beauty_test = pd.read_csv(r'./SequentialRecommend01/data/Amazon_Beauty/test_sessions.csv')

In [131]:
NARM_cellphone_model = torch.load(r'./SequentialRecommend01/model/NARM_cellphone_150.pth')

In [132]:
NARM_beauty_model = torch.load(r'./SequentialRecommend01/model/NARM_beauty_180.pth')

In [133]:
NARM_cellphone_model.eval()
sims = 0.0
count = 0
topk = 20
#recalls = []
#mrrs = []

df_cellphone_all = []
with torch.no_grad():
    df_cellphhone_test.reset_index(drop=True, inplace=True)
    
        
    for batch in utils.batch_loader(df_cellphhone_test.index.tolist(), 128):

        df_chunk = df_cellphhone_test.loc[batch,:]

        index_ = len_argsort(df_chunk.session.tolist())
        
        df_chunk.reset_index(drop=True, inplace=True)
        items_seqs, _, items_seqs_len, _ = preprocess.sequence_(df_chunk.loc[index_,:],
                                                                     item_key='session',
                                                                     next_key='session')
        items_seqs = utils.to_tensor(items_seqs).T
        #print(items_seqs.shape)
        items_seqs_len = utils.to_tensor(items_seqs_len).to(torch.device('cpu'))
        #items_next = utils.to_tensor(items_next)
        #print(items_next.shape)
        #batch[negative_seqs_name] = neg_next
        outputs = NARM_cellphone_model(items_seqs, items_seqs_len)
        """logits = F.softmax(outputs, dim = 1)
        #print(logits.shape, items_next.shape)
        recall, mrr = metric.evaluate(logits, items_next, k = topk)
        recalls.append(recall)
        mrrs.append(mrr)"""
        
        predicts = outputs.topk(topk)
        #print(len(index_), predicts.indices.shape)
        
        predicts = predicts.indices.cpu().numpy()
        predicts = [str(list(predict)) for predict in predicts]
        
        #df_chunk.loc[index_,'predict']  = np.arange(len(index_))
        df_chunk.loc[index_,'predict'] = predicts
        df_chunk.loc[index_,'predict']  = df_chunk.predict.astype(str)
        df_cellphone_all.append(df_chunk)
df_cellphone_all = pd.concat(df_cellphone_all, ignore_index=True).drop_duplicates(keep = 'first')

In [134]:
print(len(df_cellphone_all))
print(len(df_cellphhone_test))

3086
3915


In [135]:
df_cellphhone_test = pd.merge(df_cellphhone_test, df_cellphone_all, how='left', on='session')
df_cellphhone_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3915 entries, 0 to 3914
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   session  3915 non-null   object
 1   predict  3915 non-null   object
dtypes: object(2)
memory usage: 91.8+ KB


In [136]:
df_cellphhone_test.predict.isnull().sum()

0

In [137]:
NARM_beauty_model.eval()
sims = 0.0
count = 0
topk = 20
df_beauty_all = []
with torch.no_grad():
    df_beauty_test.reset_index(drop=True, inplace=True)
    
        
    for batch in utils.batch_loader(df_beauty_test.index.tolist(), 128):

        df_chunk = df_beauty_test.loc[batch,:]

        index_ = len_argsort(df_chunk.session.tolist())

        df_chunk.reset_index(drop=True, inplace=True)
        items_seqs, _, items_seqs_len, _ = preprocess.sequence_(df_chunk.loc[index_,:],
                                                                     item_key='session',
                                                                     next_key='session')
        items_seqs = utils.to_tensor(items_seqs).T
        #print(items_seqs.shape)
        items_seqs_len = utils.to_tensor(items_seqs_len).to(torch.device('cpu'))
        #items_next = utils.to_tensor(items_next)
        #print(items_next.shape)
        #batch[negative_seqs_name] = neg_next
        outputs = NARM_beauty_model(items_seqs, items_seqs_len)
        """logits = F.softmax(outputs, dim = 1)
        #print(logits.shape, items_next.shape)
        recall, mrr = metric.evaluate(logits, items_next, k = topk)
        recalls.append(recall)
        mrrs.append(mrr)"""
        predicts = outputs.topk(topk)
        
        
        predicts = predicts.indices.cpu().numpy()
        predicts = [str(list(predict)) for predict in predicts]
        
        #df_chunk.loc[index_,'predict']  = np.arange(len(index_))
        #print(len(index_), len(predicts))
        df_chunk.loc[index_,'predict'] = predicts
        df_chunk.loc[index_,'predict']  = df_chunk.predict.astype(str)
        df_beauty_all.append(df_chunk)
df_beauty_all = pd.concat(df_beauty_all, ignore_index=True).drop_duplicates(keep = 'first')

In [138]:
print(len(df_beauty_all))
print(len(df_beauty_test))

4567
5393


In [139]:
df_beauty_test = pd.merge(df_beauty_test, df_beauty_all, how='left', on='session')
df_beauty_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5393 entries, 0 to 5392
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   session  5393 non-null   object
 1   predict  5393 non-null   object
dtypes: object(2)
memory usage: 126.4+ KB


In [140]:
df_cellphhone_test.to_csv(r'./SequentialRecommend01/data/cellphone_predict.csv', index=False)
df_beauty_test.to_csv(r'./SequentialRecommend01/data/beauty_predict.csv', index=False)